In [1]:
import pandas as pd
import numpy as np
import sys
# sys.path.append("/home/javonne/irp-jp2923/src")
sys.path.append("/home/javonne/severity_or_subtype/src")

import functions_data
import functions_abide

In [2]:
df = pd.read_csv(
    # "../data/abide_ii/ABIDEII-PERSONALISED_COMPOSITE.csv",
    "../data/abide_ii/ABIDE_II_SEQUENTIAL_STRUCTURAL_PHENOTYPIC.csv",
    
    index_col="SUB_ID"
)

df.head()


,SITE_ID,NDAR_GUID,DX_GROUP,PDD_DSM_IV_TR,ASD_DSM_5,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,...,ADI_R_C3_TOTAL,ADI_R_C4_REPETITIVE_USE_OBJECTS,ADI_R_C4_HIGHER,ADI_R_C4_UNUSUAL_SENSORY_INTERESTS,ADI_R_C4_TOTAL,ADI_R_D_AGE_PARENT_NOTICED,ADI_R_D_AGE_FIRST_SINGLE_WORDS,ADI_R_D_AGE_FIRST_PHRASES,ADI_R_D_AGE_WHEN_ABNORMALITY,ADI_R_D_INTERVIEWER_JUDGMENT
SUB_ID,,,,,,,,,,,,,,,,,,,,,
sub-001,ABIDEII-BNI_1,NaN,1,NaN,NaN,48.0,1,1,NaN,131.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-002,ABIDEII-BNI_1,NaN,1,NaN,NaN,41.0,1,1,NaN,110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-003,ABIDEII-BNI_1,NaN,1,NaN,NaN,59.0,1,1,NaN,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-004,ABIDEII-BNI_1,NaN,1,NaN,NaN,57.0,1,1,NaN,114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-005,ABIDEII-BNI_1,NaN,1,NaN,NaN,45.0,1,1,NaN,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# initial handling
Remove underpopulated features, duplicates etc

In [3]:
# drop duplicates
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))


376
376


In [4]:
# This cell was written by me during the DSML module of the ACSE course.
# 1 removing the features with >30% nans

# find cols to remove
cols_to_remove = [column for column in df.columns if functions_abide.nan_ratio_col(df, column) > 0.15]
df.drop(columns=cols_to_remove, inplace=True, axis=1)

In [5]:
df.columns

Index(['SITE_ID', 'DX_GROUP', 'AGE_AT_SCAN ', 'SEX', 'HANDEDNESS_CATEGORY',
       'FIQ', 'FIQ_TEST_TYPE', 'SRS_EDITION', 'SRS_VERSION', 'SRS_INFORMANT',
       'SRS_TOTAL_RAW', 'SRS_AWARENESS_RAW', 'SRS_COGNITION_RAW',
       'SRS_COMMUNICATION_RAW', 'SRS_MOTIVATION_RAW', 'SRS_MANNERISMS_RAW',
       'SRS_TOTAL_T', 'SRS_AWARENESS_T', 'SRS_COGNITION_T',
       'SRS_COMMUNICATION_T', 'SRS_MOTIVATION_T', 'SRS_MANNERISMS_T',
       'CURRENT_MED_STATUS', 'CURRENT_MEDICATION_NAME', 'EYE_STATUS_AT_SCAN'],
      dtype='object')

In [6]:
drop_cols=[

    "SRS_INFORMANT",
    "EYE_STATUS_AT_SCAN",
    "CURRENT_MEDICATION_NAME",
    "HANDEDNESS_CATEGORY",# consider keeping???? idk.
    "FIQ_TEST_TYPE",# consider keeping???? idk.
    # drop the T values because we have raw values
    "SRS_TOTAL_T","SRS_AWARENESS_T", "SRS_COGNITION_T","SRS_COMMUNICATION_T","SRS_MOTIVATION_T","SRS_MANNERISMS_T",
    # drop raw srs because we calculate out own total!
    # "SRS_TOTAL_RAW"
]

preserved_cols=["SITE_ID","DX_GROUP","AGE_AT_SCAN ","SRS_EDITION", "SRS_VERSION", "SEX"]

In [7]:
print(df.CURRENT_MED_STATUS.value_counts())

# Med masks
mask_on_medication = (df.CURRENT_MED_STATUS != "0") & (df.CURRENT_MED_STATUS != 0)
df.loc[mask_on_medication, "CURRENT_MED_STATUS"]=1
df.CURRENT_MED_STATUS = df.CURRENT_MED_STATUS.astype(int)

CURRENT_MED_STATUS
0                   279
1                    27
ADHD combined         5
ODD                   3
300.29                3
ADHD inattentive      3
300.3                 2
ADHD                  2
300.02                2
313.81                1
past)                 1
300.23                1
dark)                 1
dark                  1
GAD                   1
296.2                 1
315.2                 1
OCD                   1
Name: count, dtype: int64


In [8]:
print(df.CURRENT_MED_STATUS.value_counts())


CURRENT_MED_STATUS
0    279
1     97
Name: count, dtype: int64


In [9]:
df.head()

,SITE_ID,DX_GROUP,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,FIQ,FIQ_TEST_TYPE,SRS_EDITION,SRS_VERSION,SRS_INFORMANT,...,SRS_MANNERISMS_RAW,SRS_TOTAL_T,SRS_AWARENESS_T,SRS_COGNITION_T,SRS_COMMUNICATION_T,SRS_MOTIVATION_T,SRS_MANNERISMS_T,CURRENT_MED_STATUS,CURRENT_MEDICATION_NAME,EYE_STATUS_AT_SCAN
SUB_ID,,,,,,,,,,,,,,,,,,,,,
sub-001,ABIDEII-BNI_1,1,48.0,1,1,131.0,KBIT-2,2.0,2.0,5.0,...,25.0,79.0,66.0,86.0,75.0,69.0,82.0,0,none,2
sub-002,ABIDEII-BNI_1,1,41.0,1,1,110.0,KBIT-2,2.0,2.0,5.0,...,19.0,65.0,52.0,62.0,68.0,56.0,72.0,1,venlafaxine,NaN
sub-003,ABIDEII-BNI_1,1,59.0,1,1,117.0,KBIT-2,2.0,2.0,5.0,...,9.0,57.0,49.0,49.0,63.0,57.0,55.0,0,none,2
sub-004,ABIDEII-BNI_1,1,57.0,1,1,114.0,KBIT-2,2.0,2.0,5.0,...,8.0,56.0,41.0,62.0,55.0,62.0,53.0,0,none,2
sub-005,ABIDEII-BNI_1,1,45.0,1,1,109.0,KBIT-2,2.0,2.0,5.0,...,28.0,87.0,86.0,83.0,86.0,76.0,87.0,0,none,2


In [10]:
# get a df with important meta info
df_special = df[preserved_cols]

# drop the columns we dont want to use/consider
df.drop(columns=drop_cols, axis=1, inplace=True)

# get a dataset to allow us to calculate allistic averages
# Thisi s the last thing we do THEREFORE df = df_autistic + df_allistic
df_autistic=df[df_special["DX_GROUP"]==1]
df_allistic=df[df_special["DX_GROUP"]==2]

In [11]:
df.columns

Index(['SITE_ID', 'DX_GROUP', 'AGE_AT_SCAN ', 'SEX', 'FIQ', 'SRS_EDITION',
       'SRS_VERSION', 'SRS_TOTAL_RAW', 'SRS_AWARENESS_RAW',
       'SRS_COGNITION_RAW', 'SRS_COMMUNICATION_RAW', 'SRS_MOTIVATION_RAW',
       'SRS_MANNERISMS_RAW', 'CURRENT_MED_STATUS'],
      dtype='object')

In [12]:
# no scalling!!
# I want to capture the true variance in the raw values!!
# Scalling is useful for regression!!!
df

,SITE_ID,DX_GROUP,AGE_AT_SCAN,SEX,FIQ,SRS_EDITION,SRS_VERSION,SRS_TOTAL_RAW,SRS_AWARENESS_RAW,SRS_COGNITION_RAW,SRS_COMMUNICATION_RAW,SRS_MOTIVATION_RAW,SRS_MANNERISMS_RAW,CURRENT_MED_STATUS
SUB_ID,,,,,,,,,,,,,,
sub-001,ABIDEII-BNI_1,1,48.00,1,131.0,2.0,2.0,122.0,12.0,28.0,38.0,19.0,25.0,0
sub-002,ABIDEII-BNI_1,1,41.00,1,110.0,2.0,2.0,82.0,7.0,14.0,31.0,11.0,19.0,1
sub-003,ABIDEII-BNI_1,1,59.00,1,117.0,2.0,2.0,60.0,6.0,7.0,26.0,12.0,9.0,0
sub-004,ABIDEII-BNI_1,1,57.00,1,114.0,2.0,2.0,58.0,3.0,14.0,18.0,15.0,8.0,0
sub-005,ABIDEII-BNI_1,1,45.00,1,109.0,2.0,2.0,146.0,19.0,26.0,50.0,23.0,28.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-372,ABIDEII-TCD_1,2,19.50,1,111.0,1.0,1.0,8.0,2.0,1.0,2.0,2.0,1.0,0
sub-373,ABIDEII-TCD_1,2,11.00,1,116.0,1.0,1.0,45.0,12.0,5.0,13.0,10.0,5.0,0
sub-374,ABIDEII-TCD_1,2,18.75,1,120.0,1.0,1.0,29.0,5.0,2.0,10.0,7.0,5.0,0


In [13]:
functions_data.save_pickle(
    filename="../data/abide_ii/phenotypic_data_asd_all.pkl", 
    object_to_pickle=df.sort_values(by='SUB_ID', ascending=True)
)

functions_data.save_pickle(filename="../data/abide_ii/phenotypic_data_special.pkl", object_to_pickle=df_special.sort_values(by='SUB_ID', ascending=True))
functions_data.save_pickle(filename="../data/abide_ii/phenotypic_data_asd.pkl", object_to_pickle=df_autistic.sort_values(by='SUB_ID', ascending=True))
functions_data.save_pickle(filename="../data/abide_ii/phenotypic_data_all.pkl", object_to_pickle=df_allistic.sort_values(by='SUB_ID', ascending=True))

In [14]:
df

,SITE_ID,DX_GROUP,AGE_AT_SCAN,SEX,FIQ,SRS_EDITION,SRS_VERSION,SRS_TOTAL_RAW,SRS_AWARENESS_RAW,SRS_COGNITION_RAW,SRS_COMMUNICATION_RAW,SRS_MOTIVATION_RAW,SRS_MANNERISMS_RAW,CURRENT_MED_STATUS
SUB_ID,,,,,,,,,,,,,,
sub-001,ABIDEII-BNI_1,1,48.00,1,131.0,2.0,2.0,122.0,12.0,28.0,38.0,19.0,25.0,0
sub-002,ABIDEII-BNI_1,1,41.00,1,110.0,2.0,2.0,82.0,7.0,14.0,31.0,11.0,19.0,1
sub-003,ABIDEII-BNI_1,1,59.00,1,117.0,2.0,2.0,60.0,6.0,7.0,26.0,12.0,9.0,0
sub-004,ABIDEII-BNI_1,1,57.00,1,114.0,2.0,2.0,58.0,3.0,14.0,18.0,15.0,8.0,0
sub-005,ABIDEII-BNI_1,1,45.00,1,109.0,2.0,2.0,146.0,19.0,26.0,50.0,23.0,28.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-372,ABIDEII-TCD_1,2,19.50,1,111.0,1.0,1.0,8.0,2.0,1.0,2.0,2.0,1.0,0
sub-373,ABIDEII-TCD_1,2,11.00,1,116.0,1.0,1.0,45.0,12.0,5.0,13.0,10.0,5.0,0
sub-374,ABIDEII-TCD_1,2,18.75,1,120.0,1.0,1.0,29.0,5.0,2.0,10.0,7.0,5.0,0


In [15]:
df_autistic

,SITE_ID,DX_GROUP,AGE_AT_SCAN,SEX,FIQ,SRS_EDITION,SRS_VERSION,SRS_TOTAL_RAW,SRS_AWARENESS_RAW,SRS_COGNITION_RAW,SRS_COMMUNICATION_RAW,SRS_MOTIVATION_RAW,SRS_MANNERISMS_RAW,CURRENT_MED_STATUS
SUB_ID,,,,,,,,,,,,,,
sub-001,ABIDEII-BNI_1,1,48.00,1,131.0,2.0,2.0,122.0,12.0,28.0,38.0,19.0,25.0,0
sub-002,ABIDEII-BNI_1,1,41.00,1,110.0,2.0,2.0,82.0,7.0,14.0,31.0,11.0,19.0,1
sub-003,ABIDEII-BNI_1,1,59.00,1,117.0,2.0,2.0,60.0,6.0,7.0,26.0,12.0,9.0,0
sub-004,ABIDEII-BNI_1,1,57.00,1,114.0,2.0,2.0,58.0,3.0,14.0,18.0,15.0,8.0,0
sub-005,ABIDEII-BNI_1,1,45.00,1,109.0,2.0,2.0,146.0,19.0,26.0,50.0,23.0,28.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-351,ABIDEII-TCD_1,1,10.75,1,139.0,1.0,1.0,69.0,14.0,13.0,28.0,8.0,6.0,0
sub-352,ABIDEII-TCD_1,1,12.75,1,120.0,1.0,1.0,105.0,13.0,19.0,42.0,12.0,19.0,0
sub-353,ABIDEII-TCD_1,1,18.75,1,119.0,1.0,1.0,65.0,9.0,15.0,24.0,8.0,9.0,0


In [16]:
df_allistic

,SITE_ID,DX_GROUP,AGE_AT_SCAN,SEX,FIQ,SRS_EDITION,SRS_VERSION,SRS_TOTAL_RAW,SRS_AWARENESS_RAW,SRS_COGNITION_RAW,SRS_COMMUNICATION_RAW,SRS_MOTIVATION_RAW,SRS_MANNERISMS_RAW,CURRENT_MED_STATUS
SUB_ID,,,,,,,,,,,,,,
sub-006,ABIDEII-BNI_1,2,48.00,1,141.0,2.0,2.0,29.0,5.0,8.0,6.0,6.0,4.0,0
sub-009,ABIDEII-BNI_1,2,64.00,1,113.0,2.0,2.0,50.0,7.0,8.0,17.0,12.0,6.0,0
sub-011,ABIDEII-BNI_1,2,41.00,1,106.0,2.0,2.0,39.0,11.0,6.0,10.0,8.0,4.0,0
sub-013,ABIDEII-BNI_1,2,51.00,1,127.0,2.0,2.0,28.0,5.0,9.0,6.0,3.0,5.0,0
sub-014,ABIDEII-BNI_1,2,62.00,1,100.0,2.0,2.0,66.0,5.0,20.0,15.0,16.0,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-372,ABIDEII-TCD_1,2,19.50,1,111.0,1.0,1.0,8.0,2.0,1.0,2.0,2.0,1.0,0
sub-373,ABIDEII-TCD_1,2,11.00,1,116.0,1.0,1.0,45.0,12.0,5.0,13.0,10.0,5.0,0
sub-374,ABIDEII-TCD_1,2,18.75,1,120.0,1.0,1.0,29.0,5.0,2.0,10.0,7.0,5.0,0
